# 4. Method Duplication

There are multiple methods in Pandas that do the exact same thing. Whenever two methods share the same exact underlying functionality, we say that they are **aliases** of each other.  Having duplication in a library is completely unnecessary and just pollutes the namespace and forces analysts to remember one more bit of information about a library.

This notebook covers several instances of duplication along with other instances of methods that are very similar to one another.

### `read_csv` vs `read_table` duplication
One example of duplication is with the `read_csv` and `read_table` functions. They both do the same exact thing, read in data from a text file. The only difference is that `read_csv` defaults the delimiter to a comma, while `read_table` has the default parameter as a tab.

Let's verify that `read_csv` and `read_table` are the same. The `equals` method verifies whether two DataFrames have the exact same values.

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 100)

college = pd.read_csv('data/college.csv')
college2 = pd.read_table('data/college.csv', delimiter=',')
college.equals(college2)

### `read_table` is getting deprecated
I made a post in [the Pandas Github repo][1] suggesting that a few functions and methods that I'd like to see deprecated. The `read_table` function is getting deprecated and should never be used.

**GUIDANCE** - Use only `read_csv`

[1]: https://github.com/pandas-dev/pandas/issues/18262#issuecomment-346502617

### `isna` vs `isnull` and `notna` vs `notnull`
The `isna` and `isnull` methods both determine whether each value in the DataFrame is missing or not. The result will always be a DataFrame (or Series) of all boolean values.

These methods are exactly the same. We say that one is an **alias** for the other. There is no need for both of them in the library. The `isna` method was added more recently because the characters `na` are found in other missing value methods such as `dropna` and `fillna`.

`notna` and `notnull` are aliases of each other as well and simply return the opposite of `isna`. There's no need for both of them.

Let's verify that `isna` and `isnull` are aliases.

In [ ]:
college_isna = college.isna()
college_isnull = college.isnull()
college_isna.equals(college_isnull)

### I only use `isna` and `notna`

I use the methods that end in `na` to match the names of the other missing value methods `dropna` and `fillna`. 

You can also avoid ever using `notna` since Pandas provides the inversion operator, `~` to invert boolean DataFrames.

**GUIDANCE** - Use only `isna` and `notna`

## Arithmetic and Comparison Operators

All arithmetic operators have corresponding methods that function similarly.

* `+` - `add`
* `-` - `sub` and `subtract`
* `*` - `mul` and `multiply`
* `/` - `div`, `divide` and `truediv`
* `/` - `pow`
* `//` - `floordiv`
* `%` - `mod`

All the comparison operators also have corresponding methods.

* `>` - `gt`
* `<` - `lt`
* `>=` - `ge`
* `<=` - `le`
* `==` - `eq`
* `!=` - `ne`

Let's select the undergraduate population (ugds) column as Series, add 100 to it and verify that both the plus operator its corresponding method, `add` give the same result.

In [ ]:
ugds = college['ugds']
ugds_operator = ugds + 100
ugds_method = ugds.add(100)
ugds_operator.equals(ugds_method)

### Calculating the z-scores of each school
Let's do a slightly more complex example. Below, we set the index to be the institution name and then select both of the SAT columns.

In [ ]:
college_idx = college.set_index('instnm')
sats = college_idx[['satmtmid', 'satvrmid']].dropna()
sats.head()

Let's say we are interested in finding the z-score for each college's SAT score. To calculate this, we would need to subtract the mean and divide by the standard deviation. Let's do that first with operators.

In [ ]:
mean = sats.mean()
mean

In [ ]:
std = sats.std()
std

In [ ]:
zscore_operator = (sats - mean) / std
zscore_operator.head()

Let's repeat this with the methods and verify equality.

In [ ]:
zscore_methods = sats.sub(mean).div(std)
zscore_operator.equals(zscore_methods)

### An actual need for the method
So far we haven't seen an explicit need for the methods over the operators. Let's see an example where we absolutely need the method to complete the task. The college dataset contains 9 consecutive columns holding the frequency of the undergraduate population by race. The first column is `ugds_white` and the last `ugds_unkn`. Let's select these columns now into their own DataFrame.

In [ ]:
college_race = college_idx.loc[:, 'ugds_white':'ugds_unkn']
college_race.head()

Let's say we are interested in the raw count of each race per school. We would need to multiply the total undergraduate population by each column. Below we select the `ugds` column as a Series.

In [ ]:
ugds = college_idx['ugds']
ugds.head()

We then multiple the `college_race` DataFrame by this Series. Intuitively, this seems like it should work, but it doesn't. Instead, it returns an enormous DataFrame with 7,544 columns.

In [ ]:
df_attempt = college_race * ugds
df_attempt.head()

In [ ]:
df_attempt.shape

### Automatic alignment on the index and/or columns
Whenever an operation takes place between two Pandas objects, an alignment always takes place between the index and/or columns of the two objects. In the above operation, we multiplied The `college_race` **DataFrame** and the `ugds` **Series** together. Pandas automatically (implicitly) aligned the **columns** of `college_race` to the **index** values of `ugds`.

None of the `college_race` columns match the index values of `ugds`. Pandas does the alignment by performing an **outer join** keeping all values that match as well as those that do not. This returns a ridiculous looking DataFrame with all missing values. Scroll all the way to the right to view the original column names of the `college_race` DataFrame.

### Change the direction of the alignment with a method
All operators only work in a single way. We cannot change how the multiplication operator, `*`, works. Methods, on the other hand, can have parameters that we can use to control how the operation takes place. 

### Use the `axis` parameter of the `mul` method
All the methods that correspond to the operators listed above have an `axis` parameter that allows us to change the direction of the alignment. So, instead of aligning the columns of a DataFrame to the index of a Series, we can align the index of a DataFrame to the index of a Series. Let's do that now so that we can find the answer to our problem from above.

In [ ]:
df_correct = college_race.mul(ugds, axis='index').round(0)
df_correct.head()

### Discussion
By default, the `axis` parameter is set to 'columns'. We changed it to 'index' so that a proper alignment took place

### Another use-case for the method - filling in missing values
There exists another use-case when the operator cannot be used. Take a look at the following two Series. They are each a different length and have some index values in common.

In [ ]:
s1 = pd.Series(index=['a', 'b', 'c', 'e'], data=[4, 8, 3, 5])
s2 = pd.Series(index=['a', 'b', 'd'], data=[2, 1, 9])

In [ ]:
s1

In [ ]:
s2

It is still possible to do an arithmetic or comparison operation between them. Let's add the two Series together. 

In [ ]:
s1 + s2

### Explanation
Pandas automatically aligns the indexes together and then adds the values of the Series. Only index 'a' and 'b' are contained in each Series. The others don't align, but are still kept in the result. Again, Pandas does an outer join here when aligning.

### Use the corresponding method to fill in the missing values
The `add` method performs the same addition but allows us to control what happens to those values that don't align. We can set the value to be used if there is no alignment with the `fill_value` parameter. Below, we use it to add 100 to each value that is not aligned.

In [ ]:
s1.add(s2, fill_value=100)

## Built in functions vs Pandas methods with the same name
There are a few DataFrame/Series methods that will return the same result if a built-in Python function with the same name is used. They are:
* `sum`
* `min`
* `max`
* `abs`

Let's verify that the give the same result by testing them out on a single column of data.

In [ ]:
ugds = college['ugds'].dropna()

In [ ]:
sum(ugds)

In [ ]:
ugds.sum()

In [ ]:
max(ugds)

In [ ]:
ugds.max()

In [ ]:
min(ugds)

In [ ]:
ugds.min()

In [ ]:
abs(ugds).head()

In [ ]:
ugds.abs().head()

### Time the performance of each

**`sum`**

In [ ]:
%timeit -n 5 sum(ugds)

In [ ]:
%timeit -n 5 ugds.sum()

**`min`**

In [ ]:
%timeit -n 5 min(ugds)

In [ ]:
%timeit -n 5 ugds.min()

**`max`**

In [ ]:
%timeit -n 5 max(ugds)

In [ ]:
%timeit -n 5 ugds.max()

**`abs`**

In [ ]:
%timeit -n 5 abs(ugds)

In [ ]:
%timeit -n 5 ugds.abs()

## Performance discrepancy for `sum`, `max`, and `min`
There are clear performance discrepancies for `sum`, `max`, and `min`. Completely different code is executed when the built-in Python functions are used as opposed to when the Pandas method is called. Calling `sum(ugds)` essentially creates a Python for loop to iterate through each value one at a time. On the other hand, calling `ugds.sum()` executes the internal Pandas `sum` method which is written in C and much faster than the iterating with a Python for loop.

There is a lot of overhead in Pandas which is why the difference is not greater. If we instead create a NumPy array and redo the timings, we can see an enormous difference with the Numpy array `sum` outperforming the Python `sum` function by a factor of 200 on an array of 10,000 floats.

In [ ]:
len(ugds)

In [ ]:
import numpy as np

In [ ]:
a = np.random.rand(10**4)

In [ ]:
%timeit -n 10 sum(a)

In [ ]:
%timeit -n 100 a.sum()

## No Performance difference for `abs`
Notice that there is no performance difference when calling the `abs` function versus the `abs` Pandas method. This is because the exact same underlying code is being called. This is due to how Python chose to design the `abs` function. It allows developers to provide a custom `abs` method to be executed whenever the `abs` function is called. So, they are both literally the same.

**GUIDANCE** - Use the Pandas method over any built-in Python function with the same name.